In [0]:
import pandas as pd
import statsmodels.api as sm
import numpy as np
from pyspark.sql.types import *
from pyspark.sql.functions import pandas_udf, PandasUDFType

In [0]:
# criando um dataframe
pdf = pd.DataFrame({'group_id':[1,1,1,1,1,1,2,2,2,2,2,2,3,3,3,3,3,3,3,3,4,4,4,4,4,4],
                    'sex':['M','N','M','N','F','F','M','F','N','M','M','N','F','M','F','F','N','M','F','M','F','N','M','F','M','F'],
                    'x':[0,1,2,0,1,5,2,3,4,5,6,2,3,4,1,2,6,7,8,5,3,4,1,7,6,5],
                    'y':[2,1,0,0,0,5,2,5,3,4,5,6,1,2,5,6,7,8,9,4,2,5,8,10,5,6]})
df = sqlContext.createDataFrame(pdf)

#visualizando 
df.display()

group_id,sex,x,y
1,M,0,2
1,N,1,1
1,M,2,0
1,N,0,0
1,F,1,0
1,F,5,5
2,M,2,2
2,F,3,5
2,N,4,3
2,M,5,4


In [0]:
# schema do resultado
result_schema =StructType([
  StructField('group_id',DoubleType()),
  StructField('sex',StringType()),
  StructField('x',DoubleType())
 ])

In [0]:
# criando o pandas udf para regreções
@pandas_udf(result_schema, PandasUDFType.GROUPED_MAP)
def ols(df):
    group_id = df['group_id'].iloc[0]
    sex = df['sex'].iloc[0]

    if len(df) == 1:
        return pd.DataFrame([[group_id] + [sex] + [None]], columns=['group_id'] + ['sex'] + ['x'])

    else:        
        y = df['y'].astype(int)
        X = df['x'].astype(int)
        X = sm.add_constant(X)
        model = sm.OLS(y, X).fit()
        return pd.DataFrame([[group_id] + [sex] + [model.params[1]]], columns=['group_id'] + ['sex'] + ['x'])


# agrupando a base e aplicando as regressões (captando o BETA)
df.groupby('group_id', 'sex').apply(ols).display()

/databricks/spark/python/pyspark/sql/pandas/group_ops.py:76: UserWarning: It is preferred to use 'applyInPandas' over this API. This API will be deprecated in the future releases. See SPARK-28264 for more details.
 "more details.", UserWarning)

group_id,sex,x
1.0,F,1.2500000000000002
1.0,M,-0.9999999999999998
1.0,N,1.0000000000000004
2.0,F,null
2.0,M,0.7307692307692304
2.0,N,-1.5
3.0,F,0.6379310344827591
3.0,M,2.0000000000000013
3.0,N,null
4.0,F,2.0000000000000004
